In [21]:
import pandas as pd
import numpy as np
import os
import shutil

In [22]:
from sqlalchemy import create_engine
engine = create_engine(r'sqlite:///C:\Users\njones\Documents\addresses.sqlite', echo=False)

In [23]:
header_list = [
    'ROW_NUM','INPUT_ADDRESS','MATCH','MATCH_TYPE','TIGER_OUTPUT_ADDRESS','LONLAT','TIGERLINE_ID','TIGERLINE_ID_SIDE'
]

#.loc[row_indexer,col_indexer]

In [24]:
for root, dirs, files in os.walk(r"C:\Users\njones\Documents\Python Scripts\Census Geocoding\downloaded_csvs"):
    for filename in files: #[:2]:
        print(root+"\\"+filename)
        df = pd.read_csv(r"%s"%(root+"\\"+filename), names=header_list)
        matches_df = df.loc[df['MATCH']=='Match']
        non_matches_df = df.loc[(df['MATCH']=='No_Match')|(df['MATCH']=='Tie')]

        matches_df[['STREET_ADDRESS', 'CITY', 'STATE','ZIP_CODE']] = matches_df['INPUT_ADDRESS'].str.split(",", expand=True)[[0,1,2,3]]
        non_matches_df[['STREET_ADDRESS', 'CITY', 'STATE','ZIP_CODE']] = non_matches_df['INPUT_ADDRESS'].str.split(",", expand=True)[[0,1,2,3]]

        non_matches_df[['O_ADDRESS', 'O_CITY', 'O_STATE','O_ZIP_CODE']] = ""
        matches_df[['O_ADDRESS', 'O_CITY', 'O_STATE','O_ZIP_CODE']] = matches_df['TIGER_OUTPUT_ADDRESS'].str.split(",", expand=True)[[0,1,2,3]]

        
        matches_df[['LON', 'LAT']] = matches_df['LONLAT'].str.split(",", expand=True)
        
        matches_df[['filename']] = filename
        non_matches_df[['filename']] = filename
        
        # Trim all non-numeric cloumn values of extra characters
        for c in matches_df.select_dtypes(exclude=[np.number]).columns.tolist():
            matches_df[c] = matches_df[c].str.strip()
            
        for c in non_matches_df.select_dtypes(exclude=[np.number]).columns.tolist():
            non_matches_df[c] = non_matches_df[c].str.strip()    
            

        # Write dataframe to the SQLite database
        matches_df.to_sql('address_coords', con=engine, index=False, if_exists='append') 
        non_matches_df.to_sql('address_coords', con=engine, index=False, if_exists='append') 
        
        dest = shutil.move(
            root+"\\"+filename,
            "C:\\Users\\njones\\Documents\\Python Scripts\\Census Geocoding\\inserted_csvs\\"+filename
            ) 

C:\Users\njones\Documents\Python Scripts\Census Geocoding\downloaded_csvs\output_641_d.csv


C:\Users\njones\AppData\Local\Temp/ipykernel_12492/1779074961.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  matches_df[c] = matches_df[c].str.strip()
C:\Users\njones\AppData\Local\Temp/ipykernel_12492/1779074961.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  non_matches_df[c] = non_matches_df[c].str.strip()


C:\Users\njones\Documents\Python Scripts\Census Geocoding\downloaded_csvs\output_642_d.csv


In [5]:
non_matches_df

,ROW_NUM,INPUT_ADDRESS,MATCH,MATCH_TYPE,TIGER_OUTPUT_ADDRESS,LONLAT,TIGERLINE_ID,TIGERLINE_ID_SIDE,STREET_ADDRESS,CITY,STATE,ZIP_CODE,O_ADDRESS,O_CITY,O_STATE,O_ZIP_CODE,filename
3,2306,"P O BOX 101, WENDEL, PA, 15691",No_Match,NaN,NaN,NaN,NaN,NaN,P O BOX 101,WENDEL,PA,15691,,,,,output_5_d.csv
6,2305,"210 E PINE ST, GROVE CITY, PA, 16127",Tie,NaN,NaN,NaN,NaN,NaN,210 E PINE ST,GROVE CITY,PA,16127,,,,,output_5_d.csv
8,2304,"3376 BRODHEAD ROAD, ALIQUIPPA, PA, 15001",No_Match,NaN,NaN,NaN,NaN,NaN,3376 BRODHEAD ROAD,ALIQUIPPA,PA,15001,,,,,output_5_d.csv
9,2303,"4 W MOODY AVE, NEW CASTLE, PA, 16101",Tie,NaN,NaN,NaN,NaN,NaN,4 W MOODY AVE,NEW CASTLE,PA,16101,,,,,output_5_d.csv
17,3632,"PO BOX 752, VANDERBILT, PA, 15486",No_Match,NaN,NaN,NaN,NaN,NaN,PO BOX 752,VANDERBILT,PA,15486,,,,,output_5_d.csv
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9946,4939,"PO BOX 129, MARROWBONE, KY, 42759",No_Match,NaN,NaN,NaN,NaN,NaN,PO BOX 129,MARROWBONE,KY,42759,,,,,output_5_d.csv
9957,3605,", , , 28779",No_Match,NaN,NaN,NaN,NaN,NaN,,,,28779,,,,,output_5_d.csv
9962,4948,"808 SALT RIVER RD, LEITCHFIELD, KY, 42754",No_Match,NaN,NaN,NaN,NaN,NaN,808 SALT RIVER RD,LEITCHFIELD,KY,42754,,,,,output_5_d.csv
9981,4958,"3 ROAD 1738, FARMINGTON, NM, 87401",No_Match,NaN,NaN,NaN,NaN,NaN,3 ROAD 1738,FARMINGTON,NM,87401,,,,,output_5_d.csv


In [50]:
matches_df['TIGER_OUTPUT_ADDRESS'].str.split(",", expand=True)[[0,1,2,3]]

,0,1,2,3
0,218 FAIRMONT DR,RINCON,GA,31326
1,242 E RALSTON AVE,AKRON,OH,44301
2,110 ROSE RIDGE DR,CANTON,GA,30115
3,342 7TH AVE N,TIERRA VERDE,FL,33715
4,427 S MOUND ST,MUNCIE,IN,47305
...,...,...,...,...
9995,719 CHIMNEY ROCK RD,GREENSBORO,NC,27409
9996,409 1ST AVE,JESSUP,PA,18434
9997,1333 KERSEY VALLEY RD,ARCHDALE,NC,27263
9998,802 N HOWARD ST,AKRON,OH,44310


In [34]:
#df.to_sql('answers', con=engine, index=False, if_exists='append') 
header_list = [
    'ROW_NUM','INPUT_ADDRESS','MATCH','MATCH_TYPE','TIGER_OUTPUT_ADDRESS','LONLAT','TIGERLINE_ID','TIGERLINE_ID_SIDE'
]
df = pd.read_csv(r"C:\Users\njones\Documents\Python Scripts\Census Geocoding\downloaded_csvs\output_103_d.csv", names=header_list)
df

,ROW_NUM,INPUT_ADDRESS,MATCH,MATCH_TYPE,TIGER_OUTPUT_ADDRESS,LONLAT,TIGERLINE_ID,TIGERLINE_ID_SIDE
0,4970,"803 FIEGEL ST APT 9, SULPHUR, LA, 70663",Match,Exact,"803 FIEGEL ST, SULPHUR, LA, 70663","-93.38392,30.241407",88446157.0,R
1,3640,"202 E HILLSIDE DR, LOCK HAVEN, PA, 17745",Match,Exact,"202 HILLSIDE DR, LOCK HAVEN, PA, 17745","-77.4211,41.147392",77150088.0,L
2,4971,"155 MARY DR, WEWAHITCHKA, FL, 32465",No_Match,NaN,NaN,NaN,NaN,NaN
3,2306,"335 PEACEFUL WAY, FAYETTEVILLE, GA, 30214",Match,Exact,"335 PEACEFUL WAY, FAYETTEVILLE, GA, 30214","-84.535484,33.5129",617094865.0,R
4,3638,"32 CURTIS CIR, MILL HALL, PA, 17751",Match,Exact,"32 CURTIS CIR, MILL HALL, PA, 17751","-77.49882,41.106964",77161645.0,R
...,...,...,...,...,...,...,...,...
9995,4955,"3644 LARK LANE, PANAMA CITY, FL, 32404",Match,Exact,"3644 LARK LN, PANAMA CITY, FL, 32404","-85.60452,30.199724",7067126.0,L
9996,3628,"140 FREEDOM LN, LOCK HAVEN, PA, 17745",No_Match,NaN,NaN,NaN,NaN,NaN
9997,4956,"1217 BRITTON RD, LYNN HAVEN, FL, 32444",Match,Exact,"1217 BRITTON RD, LYNN HAVEN, FL, 32444","-85.62384,30.23905",614887951.0,R
9998,3627,"1133 E VALLEY RD, LOGANTON, PA, 17747",Tie,NaN,NaN,NaN,NaN,NaN


In [43]:
matches_df = df.loc[df['MATCH']=='Match']
non_matches_df = df.loc[(df['MATCH']=='No_Match')|(df['MATCH']=='Tie')]

matches_df[['STREET_ADDRESS', 'CITY', 'STATE','ZIP_CODE']] = matches_df['TIGER_OUTPUT_ADDRESS'].str.split(",", expand=True)
non_matches_df[['STREET_ADDRESS', 'CITY', 'STATE','ZIP_CODE']] = non_matches_df['INPUT_ADDRESS'].str.split(",", expand=True)

matches_df[['LON', 'LAT']] = matches_df['LONLAT'].str.split(",", expand=True)

matches_df
#matches_df[['STREET_ADDRESS', 'CITY', 'STATE','ZIP_CODE']] = matches_df['INPUT_ADDRESS'].str.split(",", expand=True)
#non_matches_df[['STREET_ADDRESS', 'CITY', 'STATE','ZIP_CODE']] = non_matches_df['TIGER_OUTPUT_ADDRESS'].str.split(",", expand=True)

C:\Users\njones\Anaconda3\envs\ollies\lib\site-packages\pandas\core\frame.py:3641: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


,ROW_NUM,INPUT_ADDRESS,MATCH,MATCH_TYPE,TIGER_OUTPUT_ADDRESS,LONLAT,TIGERLINE_ID,TIGERLINE_ID_SIDE,STREET_ADDRESS,CITY,STATE,ZIP_CODE,LON,LAT
0,4970,"803 FIEGEL ST APT 9, SULPHUR, LA, 70663",Match,Exact,"803 FIEGEL ST, SULPHUR, LA, 70663","-93.38392,30.241407",88446157.0,R,803 FIEGEL ST,SULPHUR,LA,70663,-93.38392,30.241407
1,3640,"202 E HILLSIDE DR, LOCK HAVEN, PA, 17745",Match,Exact,"202 HILLSIDE DR, LOCK HAVEN, PA, 17745","-77.4211,41.147392",77150088.0,L,202 HILLSIDE DR,LOCK HAVEN,PA,17745,-77.4211,41.147392
3,2306,"335 PEACEFUL WAY, FAYETTEVILLE, GA, 30214",Match,Exact,"335 PEACEFUL WAY, FAYETTEVILLE, GA, 30214","-84.535484,33.5129",617094865.0,R,335 PEACEFUL WAY,FAYETTEVILLE,GA,30214,-84.535484,33.5129
4,3638,"32 CURTIS CIR, MILL HALL, PA, 17751",Match,Exact,"32 CURTIS CIR, MILL HALL, PA, 17751","-77.49882,41.106964",77161645.0,R,32 CURTIS CIR,MILL HALL,PA,17751,-77.49882,41.106964
5,4969,"190 CHAPEL LN, PORT SAINT JOE, FL, 32456",Match,Exact,"190 CHAPEL LN, PORT SAINT JOE, FL, 32456","-85.37655,30.008272",7149788.0,L,190 CHAPEL LN,PORT SAINT JOE,FL,32456,-85.37655,30.008272
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9993,4954,"149 SIMS AVE LOT A3, PANAMA CITY, FL, 32404",Match,Exact,"149 SIMS AVE, PANAMA CITY, FL, 32404","-85.59746,30.148539",7066400.0,R,149 SIMS AVE,PANAMA CITY,FL,32404,-85.59746,30.148539
9994,3629,"1024 WEST 4TH STREET, LOCK HAVEN, PA, 17745",Match,Exact,"1024 W 4TH ST, LOCK HAVEN, PA, 17745","-77.466484,41.13153",77151426.0,R,1024 W 4TH ST,LOCK HAVEN,PA,17745,-77.466484,41.13153
9995,4955,"3644 LARK LANE, PANAMA CITY, FL, 32404",Match,Exact,"3644 LARK LN, PANAMA CITY, FL, 32404","-85.60452,30.199724",7067126.0,L,3644 LARK LN,PANAMA CITY,FL,32404,-85.60452,30.199724
9997,4956,"1217 BRITTON RD, LYNN HAVEN, FL, 32444",Match,Exact,"1217 BRITTON RD, LYNN HAVEN, FL, 32444","-85.62384,30.23905",614887951.0,R,1217 BRITTON RD,LYNN HAVEN,FL,32444,-85.62384,30.23905


In [23]:
df[['STREET_ADDRESS', 'CITY', 'STATE','ZIP_CODE']] = df['TIGER_OUTPUT_ADDRESS'].str.split(",", expand=True)
df.loc[df['TIGER_OUTPUT_ADDRESS'].isna()][['STREET_ADDRESS', 'CITY', 'STATE','ZIP_CODE']] = df.loc[df['TIGER_OUTPUT_ADDRESS'].isna()]['INPUT_ADDRESS'].str.split(",", expand=True)

df[['LON', 'LAT']] = df['LONLAT'].str.split(",", expand=True)
df

C:\Users\njones\Anaconda3\envs\ollies\lib\site-packages\pandas\core\frame.py:3641: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


,ROW_NUM,INPUT_ADDRESS,MATCH,MATCH_TYPE,TIGER_OUTPUT_ADDRESS,LONLAT,TIGERLINE_ID,TIGERLINE_ID_SIDE,STREET_ADDRESS,CITY,STATE,ZIP_CODE,LON,LAT
0,4970,"803 FIEGEL ST APT 9, SULPHUR, LA, 70663",Match,Exact,"803 FIEGEL ST, SULPHUR, LA, 70663","-93.38392,30.241407",88446157.0,R,803 FIEGEL ST,SULPHUR,LA,70663,-93.38392,30.241407
1,3640,"202 E HILLSIDE DR, LOCK HAVEN, PA, 17745",Match,Exact,"202 HILLSIDE DR, LOCK HAVEN, PA, 17745","-77.4211,41.147392",77150088.0,L,202 HILLSIDE DR,LOCK HAVEN,PA,17745,-77.4211,41.147392
2,4971,"155 MARY DR, WEWAHITCHKA, FL, 32465",No_Match,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2306,"335 PEACEFUL WAY, FAYETTEVILLE, GA, 30214",Match,Exact,"335 PEACEFUL WAY, FAYETTEVILLE, GA, 30214","-84.535484,33.5129",617094865.0,R,335 PEACEFUL WAY,FAYETTEVILLE,GA,30214,-84.535484,33.5129
4,3638,"32 CURTIS CIR, MILL HALL, PA, 17751",Match,Exact,"32 CURTIS CIR, MILL HALL, PA, 17751","-77.49882,41.106964",77161645.0,R,32 CURTIS CIR,MILL HALL,PA,17751,-77.49882,41.106964
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,4955,"3644 LARK LANE, PANAMA CITY, FL, 32404",Match,Exact,"3644 LARK LN, PANAMA CITY, FL, 32404","-85.60452,30.199724",7067126.0,L,3644 LARK LN,PANAMA CITY,FL,32404,-85.60452,30.199724
9996,3628,"140 FREEDOM LN, LOCK HAVEN, PA, 17745",No_Match,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9997,4956,"1217 BRITTON RD, LYNN HAVEN, FL, 32444",Match,Exact,"1217 BRITTON RD, LYNN HAVEN, FL, 32444","-85.62384,30.23905",614887951.0,R,1217 BRITTON RD,LYNN HAVEN,FL,32444,-85.62384,30.23905
9998,3627,"1133 E VALLEY RD, LOGANTON, PA, 17747",Tie,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
matches_df = df.loc[~df['TIGER_OUTPUT_ADDRESS'].isna()][['STREET_ADDRESS', 'CITY', 'STATE','ZIP_CODE']]
non_matches_df = df.loc[df['TIGER_OUTPUT_ADDRESS'].isna()][['STREET_ADDRESS', 'CITY', 'STATE','ZIP_CODE']]
matches_df[['STREET_ADDRESS', 'CITY', 'STATE','ZIP_CODE']] = matches_df['INPUT_ADDRESS'].str.split(",", expand=True)
non_matches_df[['STREET_ADDRESS', 'CITY', 'STATE','ZIP_CODE']] = non_matches_df['TIGER_OUTPUT_ADDRESS'].str.split(",", expand=True)

In [5]:
df

,ROW_NUM,INPUT_ADDRESS,MATCH,MATCH_TYPE,TIGER_OUTPUT_ADDRESS,LONLAT,TIGERLINE_ID,TIGERLINE_ID_SIDE
0,4970,"4470 HIGHWAY 1464, GREENSBURG, KY, 42743",Match,Non_Exact,"4470 STATE HWY 1464, GREENSBURG, KY, 42743","-85.58672,37.232044",636747294.0,R
1,3640,"132 MICHELLE DR, ELLENBORO, NC, 28040",Match,Non_Exact,"132 MICHELE DR, ELLENBORO, NC, 28040","-81.8138,35.350388",19389491.0,R
2,4971,"3560 OLD LEXINGTON RD, CAVE CITY, KY, 42127",Match,Non_Exact,"3560 OLD LEXINGTON RD, CAVE CITY, KY, 42127","-85.91789,37.133846",648865750.0,R
3,2306,"P O BOX 101, WENDEL, PA, 15691",No_Match,NaN,NaN,NaN,NaN,NaN
4,3638,"4016 TOWERY RD, SHELBY, NC, 28150",Match,Exact,"4016 TOWERY RD, SHELBY, NC, 28150","-81.656166,35.312225",19346188.0,R
...,...,...,...,...,...,...,...,...
9995,4955,"5404 RANDOLPH GOODLUCK RD, SUMMER SHADE, KY, 4...",Match,Non_Exact,"5404 RANDOLPH GOOD LUCK RD, SUMMER SHADE, KY, ...","-85.70403,36.961807",650543784.0,R
9996,3628,"5200 AUTUMNLEAF DR, NORTH CHESTERFIELD, VA, 23234",Match,Exact,"5200 AUTUMNLEAF DR, NORTH CHESTERFIELD, VA, 23234","-77.48706,37.41758",201546095.0,L
9997,4956,"411 LITTLE BARREN RD, GREENSBURG, KY, 42743",Match,Exact,"411 LITTLE BARREN RD, GREENSBURG, KY, 42743","-85.59141,37.12423",636746503.0,L
9998,3627,"2210 BEAU LN, RICHMOND, VA, 23223",Match,Exact,"2210 BEAU LN, RICHMOND, VA, 23223","-77.40598,37.556087",106649723.0,L


In [8]:
# Select non-numeric columns
df.select_dtypes(exclude=[np.number])

,INPUT_ADDRESS,MATCH,MATCH_TYPE,TIGER_OUTPUT_ADDRESS,LONLAT,TIGERLINE_ID_SIDE
0,"4470 HIGHWAY 1464, GREENSBURG, KY, 42743",Match,Non_Exact,"4470 STATE HWY 1464, GREENSBURG, KY, 42743","-85.58672,37.232044",R
1,"132 MICHELLE DR, ELLENBORO, NC, 28040",Match,Non_Exact,"132 MICHELE DR, ELLENBORO, NC, 28040","-81.8138,35.350388",R
2,"3560 OLD LEXINGTON RD, CAVE CITY, KY, 42127",Match,Non_Exact,"3560 OLD LEXINGTON RD, CAVE CITY, KY, 42127","-85.91789,37.133846",R
3,"P O BOX 101, WENDEL, PA, 15691",No_Match,NaN,NaN,NaN,NaN
4,"4016 TOWERY RD, SHELBY, NC, 28150",Match,Exact,"4016 TOWERY RD, SHELBY, NC, 28150","-81.656166,35.312225",R
...,...,...,...,...,...,...
9995,"5404 RANDOLPH GOODLUCK RD, SUMMER SHADE, KY, 4...",Match,Non_Exact,"5404 RANDOLPH GOOD LUCK RD, SUMMER SHADE, KY, ...","-85.70403,36.961807",R
9996,"5200 AUTUMNLEAF DR, NORTH CHESTERFIELD, VA, 23234",Match,Exact,"5200 AUTUMNLEAF DR, NORTH CHESTERFIELD, VA, 23234","-77.48706,37.41758",L
9997,"411 LITTLE BARREN RD, GREENSBURG, KY, 42743",Match,Exact,"411 LITTLE BARREN RD, GREENSBURG, KY, 42743","-85.59141,37.12423",L
9998,"2210 BEAU LN, RICHMOND, VA, 23223",Match,Exact,"2210 BEAU LN, RICHMOND, VA, 23223","-77.40598,37.556087",L


In [9]:
for c in df.select_dtypes(exclude=[np.number]).columns.tolist():
    df[c] = df[c].str.strip()

In [10]:
df

,ROW_NUM,INPUT_ADDRESS,MATCH,MATCH_TYPE,TIGER_OUTPUT_ADDRESS,LONLAT,TIGERLINE_ID,TIGERLINE_ID_SIDE
0,4970,"4470 HIGHWAY 1464, GREENSBURG, KY, 42743",Match,Non_Exact,"4470 STATE HWY 1464, GREENSBURG, KY, 42743","-85.58672,37.232044",636747294.0,R
1,3640,"132 MICHELLE DR, ELLENBORO, NC, 28040",Match,Non_Exact,"132 MICHELE DR, ELLENBORO, NC, 28040","-81.8138,35.350388",19389491.0,R
2,4971,"3560 OLD LEXINGTON RD, CAVE CITY, KY, 42127",Match,Non_Exact,"3560 OLD LEXINGTON RD, CAVE CITY, KY, 42127","-85.91789,37.133846",648865750.0,R
3,2306,"P O BOX 101, WENDEL, PA, 15691",No_Match,NaN,NaN,NaN,NaN,NaN
4,3638,"4016 TOWERY RD, SHELBY, NC, 28150",Match,Exact,"4016 TOWERY RD, SHELBY, NC, 28150","-81.656166,35.312225",19346188.0,R
...,...,...,...,...,...,...,...,...
9995,4955,"5404 RANDOLPH GOODLUCK RD, SUMMER SHADE, KY, 4...",Match,Non_Exact,"5404 RANDOLPH GOOD LUCK RD, SUMMER SHADE, KY, ...","-85.70403,36.961807",650543784.0,R
9996,3628,"5200 AUTUMNLEAF DR, NORTH CHESTERFIELD, VA, 23234",Match,Exact,"5200 AUTUMNLEAF DR, NORTH CHESTERFIELD, VA, 23234","-77.48706,37.41758",201546095.0,L
9997,4956,"411 LITTLE BARREN RD, GREENSBURG, KY, 42743",Match,Exact,"411 LITTLE BARREN RD, GREENSBURG, KY, 42743","-85.59141,37.12423",636746503.0,L
9998,3627,"2210 BEAU LN, RICHMOND, VA, 23223",Match,Exact,"2210 BEAU LN, RICHMOND, VA, 23223","-77.40598,37.556087",106649723.0,L


In [10]:
query = """
        SELECT * FROM address_coords WHERE MATCH = 'Match' AND STATE = 'PA' ORDER BY ZIP_CODE
    """ #.format(cols,Q,segment)

pd.read_sql(query, engine).to_csv("PA_CUSTOMERS_WITH_GPS.csv")